In [1]:
import pandas as pd 
import numpy as np 
import psycopg2

In [2]:
conn = psycopg2.connect(
    dbname="reservas_hoteles",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432")

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("SELECT version();") 
cur.fetchone()

('PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.42.34435, 64-bit',)

Tabla ciudad

Para la tabla de ciudad necesito el nombre de ciudad, que está en el csv de reservas_hoteles_limpio 

In [6]:
df_reservas = pd.read_csv("../data/reservas_hoteles_limpio.csv")
df_reservas.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
5978,8389cbe0-27c1-4dcd-ae24-75a72f9010a5,5963,Herminio,Paniagua,herminio.paniagua@example.com,False,2025-02-05,2025-03-01,2025-03-02,40,412.82,Hotel Encanto Real,3.0,Madrid
13199,b0df6241-57e7-4e28-abc6-e0316f15a4d7,13133,Heriberto,Macías,heriberto.macías@example.com,False,2025-02-02,2025-03-01,2025-03-02,27,261.83,Hotel Maravilla Real,2.0,Madrid
4750,36236e7f-aec1-4da4-8856-e89495b6e228,4742,Nerea,Plaza,nerea.plaza@example.com,False,2025-02-08,2025-03-01,2025-03-02,14,270.28,Hotel Jardines del Rey,2.0,Madrid
11266,b6c19ff9-2b43-4467-9fc0-48bf0b8fb5e3,11217,Remedios,Navas,remedios.navas@example.com,False,2025-02-03,2025-03-01,2025-03-02,36,181.79,Hotel Vista Alegre,2.0,Madrid
10212,d940a429-0504-400a-a5ef-83e5f64f8a29,10172,Severiano,Mármol,severiano.mármol@example.com,False,2025-02-05,2025-03-01,2025-03-02,47,350.24,Hotel Mirador Real,2.0,Madrid


In [7]:
tabla_ciudad = pd.DataFrame(df_reservas["ciudad"].unique(), 
                          columns=["nombre_ciudad"])

tabla_ciudad

,nombre_ciudad
0,Madrid


In [8]:
insert_query = """
        INSERT INTO ciudad (nombre_ciudad)
        VALUES (%s)
"""

In [9]:
data_to_insert = [[row["nombre_ciudad"]] for indice, row in tabla_ciudad.iterrows()]

In [10]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

Tabla eventos 

La información para esta tabla la tenemos en el csv tabla_eventos

In [28]:
df_eventos = pd.read_csv("../data/datos_extraidos/tabla_eventos.csv")
df_eventos.sample(5)

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,inicio_evento,fin_evento,ciudad
113,IX Muestra Orfebrería Contemporánea,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015.0,CALLE CONDE DUQUE 9,11:00,Museo de Arte Contemporáneo de Madrid,2025-02-05,2025-03-23,Madrid
8,Amantes de las series,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045.0,PASEO CHOPERA 10,17:30,Espacio de Igualdad Juana Doña. Arganzuela,2025-02-19,2025-03-12,Madrid
225,Triatlón por Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28011.0,PASEO PUERTA DEL ANGEL 7,00:00,Centro Deportivo Municipal Casa de Campo,2024-09-01,2025-06-30,Madrid
90,Exposición pictórica 'Lina en el mundo de Paul...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029.0,AVENIDA MONFORTE DE LEMOS 38,08:30,Biblioteca Pública Municipal José Saramago (Fu...,2025-02-04,2025-03-25,Madrid
88,Exposición de pintura: 'Evelyn contemporary art',http://www.madrid.es/sites/v/index.jsp?vgnextc...,28049.0,Carretera M-607 Km 13 L-10,08:30,Centro Sociocultural Montecarmelo (Fuencarral ...,2025-02-02,2025-03-02,Madrid


Aparte tenemos el id_ciudad como FK. Es un id que se ha generado automáticamente y por tanto hay que traérnoslo desde SQL. Para poder obtener el id_ciudad para cada evento necesito también disponer de la ciudad del evento, por lo tanto, es una columna que también hay que sacar del escrapeo de la api a pesar de que en la tabla como tal esa columna de ciudad no vaya a aparecer, sino que vaya a aparecer la del id.

In [24]:
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudad_dict = dict(cur.fetchall()) 
ciudad_dict

{'Madrid': 1}

In [35]:
data_to_insert = []
df_evento = df_eventos[["nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "inicio_evento", "fin_evento", "organizacion", "ciudad"]].drop_duplicates()
for _, row in df_eventos.iterrows(): 
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    codigo_postal = int(codigo_postal) if pd.notna(codigo_postal) else None
    direccion = row["direccion"]
    horario = row["horario"]
    fecha_inicio = pd.to_datetime(row["inicio_evento"])
    fecha_fin = pd.to_datetime(row["fin_evento"])
    organizacion = row["organizacion"]
    ciudad = row["ciudad"]
    id_ciudad = ciudad_dict.get(ciudad)
    id_ciudad = int(id_ciudad) if pd.notna(id_ciudad) else None
    data_to_insert.append([nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad]) 

tabla_eventos = pd.DataFrame(data_to_insert, columns=["nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "inicio_evento", "fin_evento", "organizacion", "id_ciudad"])
tabla_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,inicio_evento,fin_evento,organizacion,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005.0,CALLE SAN JUSTO 5,NaN,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,1.0
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009.0,PASEO COLOMBIA 1,NaN,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro),1.0
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045.0,PLAZA LEGAZPI 8,NaN,2024-10-01,2025-06-01,Matadero Madrid,1.0
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,NaN,NaN,2024-10-14,2025-06-30,NaN,NaN
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029.0,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...,1.0


In [36]:
tabla_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  247 non-null    object        
 1   url_evento     247 non-null    object        
 2   codigo_postal  219 non-null    float64       
 3   direccion      219 non-null    object        
 4   horario        160 non-null    object        
 5   inicio_evento  247 non-null    datetime64[ns]
 6   fin_evento     247 non-null    datetime64[ns]
 7   organizacion   221 non-null    object        
 8   id_ciudad      218 non-null    float64       
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 17.5+ KB


In [37]:
len(data_to_insert)

247

In [40]:
insert_query = """
        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [39]:
conn.rollback()

Tabla hoteles 

In [ ]:
CREATE TABLE hoteles (
    id_hotel SERIAL PRIMARY KEY,
    nombre_hotel TEXT,
    competencia FLOAT,
    estrellas INT CHECK (estrellas BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);

por un lado dispongo de los hoteles de la competencia en nombre_estrellas_precio.csv y de los hoteles propios en reservas_hoteles_limpio.csv

In [58]:
df_reservas.head()
df_reservas["competencia"] = df_reservas["competencia"].replace(False, float(0))
df_reservas.head()
df_propios = df_reservas[["nombre_hotel", "estrellas", "competencia", "ciudad"]].drop_duplicates()
df_propios["estrellas"] = df_propios["estrellas"].astype("Int64")
df_propios = df_propios.drop(0, axis=0).reset_index(drop=True)
df_propios

,nombre_hotel,estrellas,competencia,ciudad
0,Hotel Monte Verde,3,0.0,Madrid
1,Hotel Brisas del Mar,3,0.0,Madrid
2,Hotel Camino del Sol,3,0.0,Madrid
3,Hotel Puerta del Cielo,3,0.0,Madrid
4,Hotel Encanto Real,3,0.0,Madrid
5,Palacio del Sol,2,0.0,Madrid
6,Hotel Jardines del Rey,2,0.0,Madrid
7,Hotel Las Estrellas,2,0.0,Madrid
8,Gran Hotel Madrid,3,0.0,Madrid
9,Hotel Torre Dorada,2,0.0,Madrid


In [60]:
df_competencia = pd.read_csv("../data/datos_extraidos/nombre_estrellas_precio.csv")
df_competencia = df_competencia[["nombre_hotel", "estrellas"]]
df_competencia["competencia"] = float(1)
df_competencia["ciudad"] = "Madrid"
df_competencia

,nombre_hotel,estrellas,competencia,ciudad
0,ibis Styles Madrid Prado,3,1.0,Madrid
1,ibis Madrid Centro las Ventas,2,1.0,Madrid
2,ibis budget Madrid Vallecas,1,1.0,Madrid
3,ibis Madrid Aeropuerto Barajas,2,1.0,Madrid
4,ibis Madrid Alcorcon Tresaguas,2,1.0,Madrid
5,ibis budget Madrid Aeropuerto,1,1.0,Madrid
6,ibis Madrid Alcobendas,2,1.0,Madrid
7,ibis budget Madrid Alcorcon Móstoles,2,1.0,Madrid
8,ibis Madrid Fuenlabrada,2,1.0,Madrid
9,ibis Madrid Alcala de Henares La Garena,3,1.0,Madrid


In [61]:
df_hoteles = pd.concat([df_propios, df_competencia], ignore_index=True)
df_hoteles

,nombre_hotel,estrellas,competencia,ciudad
0,Hotel Monte Verde,3,0.0,Madrid
1,Hotel Brisas del Mar,3,0.0,Madrid
2,Hotel Camino del Sol,3,0.0,Madrid
3,Hotel Puerta del Cielo,3,0.0,Madrid
4,Hotel Encanto Real,3,0.0,Madrid
5,Palacio del Sol,2,0.0,Madrid
6,Hotel Jardines del Rey,2,0.0,Madrid
7,Hotel Las Estrellas,2,0.0,Madrid
8,Gran Hotel Madrid,3,0.0,Madrid
9,Hotel Torre Dorada,2,0.0,Madrid


In [63]:
data_to_insert = []
df_hotel = df_hoteles[["nombre_hotel", "estrellas", "competencia", "ciudad"]]
for _, row in df_hoteles.iterrows(): 
    nombre_hotel = row["nombre_hotel"]
    estrellas = row["estrellas"]
    competencia = row["competencia"]
    ciudad = row["ciudad"]
    id_ciudad = ciudad_dict.get(ciudad)
    id_ciudad = int(id_ciudad) if pd.notna(id_ciudad) else None
    data_to_insert.append([nombre_hotel, estrellas, competencia, id_ciudad]) 

tabla_hoteles = pd.DataFrame(data_to_insert, columns=["nombre_hotel", "estrellas", "competencia", "id_ciudad"])
tabla_hoteles.head()

,nombre_hotel,estrellas,competencia,id_ciudad
0,Hotel Monte Verde,3,0.0,1
1,Hotel Brisas del Mar,3,0.0,1
2,Hotel Camino del Sol,3,0.0,1
3,Hotel Puerta del Cielo,3,0.0,1
4,Hotel Encanto Real,3,0.0,1


In [64]:
tabla_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre_hotel  29 non-null     object 
 1   estrellas     29 non-null     int64  
 2   competencia   29 non-null     float64
 3   id_ciudad     29 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.0+ KB


In [65]:
insert_query = """
        INSERT INTO hoteles (nombre_hotel, estrellas, competencia, id_ciudad)
        VALUES (%s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

cur.close()
conn.close()